# 제4 고지 : 신경망 만들기 
## STEP 50 : 미니배치를 뽑아주는 DataLoader

이전 단계에서는 `Dataset` 클래스를 만들어 통일된 인터페이스로 데이터셋을 다루게 했다면, 이제는 미니배치를 뽑아주는 `DataLoader` 클래스를 구현한다. 이를 위해 우선 `Iterator` 가 무엇인지 알아보도록 한다.

### 50.1 반복자(Iterator)란?

이름에서 알 수 있듯이 원소를 반복하여 꺼내준다.

```python
>>> t= [1,2,3]
>>> x= iter(t)
>>> next(x)
1
>>> next(x)
2
>>> next(x)
3
>>> next(x)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
StopIteration
```

구체적으로 살펴보면 다음과 같다.

- 리스트를 반복자로 변환하기 위해 `iter()` 를 사용 
- `next()` 를 이용해 원소를 차례대로 호출 

<span style='background-color : #ffdce0'>💡<b> `for` 문에서 리스트의 원소를 꺼낼때 내부적으로 반복자로 변환된다.</b></span>

파이썬에서는 다음과 같이 

1. `__iter__()` 구현하고 자기 자신(`self`) 를 반환한다.
2. `__next__()` 에서 다음 원소를 반환하도록 구현한다. 이때 반환할 원소가 없다면 `StopIteration` 을 발생시킨다. 

반복자를 직접 정의할 수 있다.

```python
class MyIterator:
  def __init__(self,max_cnt):
    self.max_cnt = max_cnt 
    self.cnt = 0
  
  def __iter__(self):
    return(self)
  
  def __next__(self):
    if self.cnt == self.max_cnt :
      raise StopIteration()
    
    self.cnt += 1 
    return self.cnt
```


In [1]:
class MyIterator:
  def __init__(self,max_cnt):
    self.max_cnt = max_cnt 
    self.cnt = 0
  
  def __iter__(self):
    return(self)
  
  def __next__(self):
    if self.cnt == self.max_cnt :
      raise StopIteration()
    
    self.cnt += 1 
    return self.cnt

obj = MyIterator(5)
for x in obj:
  print(x)

1
2
3
4
5


이제 반복자에 대해 이해했으니, `DataLoader`를 구현해본다.


```python
import math 
import random
import numpy as np 

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset = dataset # Dataset
        self.batch_size = batch_size 
        self.shuffle = shuffle # 훈련용 데이터셋을 위해 기본적으로 shuffle = True
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu

        self.reset()

    def reset(self):
        self.iteration = 0
        if self.shuffle:
            self.index = np.random.permutation(len(self.dataset))
        else:
            self.index = np.arange(len(self.dataset))

    def __iter__(self):
        return self

    def __next__(self):
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration

        i, batch_size = self.iteration, self.batch_size
        batch_index = self.index[i * batch_size:(i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]

        x = np.array([example[0] for example in batch])
        t = np.array([example[1] for example in batch])

        self.iteration += 1
        return x, t

    def next(self):
        return self.__next__()

```

### 50.2 DataLoader 사용하기

이제 `DataLoader` 를 사용하면 미니배치를 꺼내오는 일이 간단해진다. 시험삼아 학습을 가정하고 사용해보도록 하자.

```python
```



In [2]:
import sys
sys.path.append("..")

from dezero.datasets import Spiral
from dezero import DataLoader


batch_size = 10 
max_epoch = 1 

train_set = Spiral(train=True)
test_set = Spiral(train=False)
train_loader = DataLoader(train_set,batch_size)
test_loader = DataLoader(test_set,batch_size,shuffle=False) # 테스트 데이터는 평가용이므로 shuffle=False

for epoch in range(max_epoch):
    for x,t in train_loader:
        print(x.shape,t.shape)
        break
    
    for x,t in test_loader:
        print(x.shape,t.shape)
        break

(10, 2) (10,)
(10, 2) (10,)


### 50.3 accuracy 함수 구현하기
성능 평가를 위해  `accuracy` 함수를 추가한다.

```python
def accuracy(y, t):
    y, t = as_variable(y), as_variable(t)

    pred = y.data.argmax(axis=1).reshape(t.shape)
    result = (pred == t.data)
    acc = result.mean()
    return Variable(as_array(acc))

import numpy as np
import dezero.functions as F 

y = np.array([[0.2,0.8,0],[0.1,0.9,0],[0.8,0.1,0.1]])
t = np.array([1,2,0])
acc = F.accuracy(y,t)
print(acc)
# variable(0.6666666666666666)
```


### 50.4 스파이럴 데이터셋 학습 코드

이제 `DataLoader` 와 `accuracy` 를 활용해 spiral dataset 을 학습해본다.

<p align='center'>
    <img src='../assets/그림 50-1.png' align='center' width='80%'>
</p>


In [3]:
import sys
sys.path.append("..")

import math
import numpy as np
import dezero
from dezero import optimizers,DataLoader
import dezero.functions as F
from dezero.models import MLP


log_interval = 20 # 20 epoch 마다 logging

# 하이퍼 파라미터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0


# 데이터 읽기 / 모델, 옵티마이저 생성
######################################
train_set = dezero.datasets.Spiral(train=True)
test_set = dezero.datasets.Spiral(train=False)
train_loader = DataLoader(train_set,batch_size)
test_loader = DataLoader(test_set,batch_size,shuffle=False) # 테스트 데이터는 평가용이므로 shuffle=False
######################################


model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)



for epoch in range(max_epoch):
    sum_loss,sum_acc = 0,0

    ######################################
    for x,t in train_loader : # 미니배치 
    ######################################
        # 기울기 산출 / 매개변수 갱신
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        ######################################
        acc = F.accuracy(y,t) # 정확도 계산
        ######################################
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(t)
        sum_acc +=float(acc.data) * len(t) 

    # 에폭마다 학습 경과 출력
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    if epoch % log_interval == 0:
        print('epoch %d, loss %.2f, acc %.2f' % (epoch + 1, avg_loss,avg_acc))
        
    sum_loss, sum_acc = 0,0
    with dezero.no_grad():
        for x,t in test_loader:
            # 기울기 산출 / 매개변수 갱신
            y = model(x)
            loss = F.softmax_cross_entropy(y, t)
            ######################################
            acc = F.accuracy(y,t) # 정확도 계산
            ######################################

            sum_loss += float(loss.data) * len(t)
            sum_acc +=float(acc.data) * len(t) 
            
    avg_loss = sum_loss / len(test_set)
    avg_acc = sum_acc / len(test_set)
    if epoch % log_interval == 0:
        print('test loss %.2f, acc %.2f' % (avg_loss,avg_acc))

epoch 1, loss 1.09, acc 0.40
test loss 1.05, acc 0.33
epoch 21, loss 0.75, acc 0.56
test loss 0.76, acc 0.61
epoch 41, loss 0.70, acc 0.58
test loss 0.71, acc 0.61
epoch 61, loss 0.60, acc 0.68
test loss 0.62, acc 0.62
epoch 81, loss 0.44, acc 0.80
test loss 0.45, acc 0.79
epoch 101, loss 0.31, acc 0.86
test loss 0.33, acc 0.88
epoch 121, loss 0.24, acc 0.93
test loss 0.25, acc 0.91
epoch 141, loss 0.19, acc 0.94
test loss 0.22, acc 0.92
epoch 161, loss 0.17, acc 0.95
test loss 0.20, acc 0.93
epoch 181, loss 0.16, acc 0.94
test loss 0.20, acc 0.93
epoch 201, loss 0.14, acc 0.95
test loss 0.17, acc 0.94
epoch 221, loss 0.14, acc 0.95
test loss 0.16, acc 0.95
epoch 241, loss 0.13, acc 0.95
test loss 0.16, acc 0.95
epoch 261, loss 0.12, acc 0.96
test loss 0.15, acc 0.95
epoch 281, loss 0.12, acc 0.97
test loss 0.15, acc 0.96
